In [81]:
# New notebook because Folium crashed the studnet.ipynb
import pandas as pd
import numpy as np
df = pd.read_csv('data/kc_house_data.csv')

In [82]:
# Functions to help convert our object datatypes into int64 / float
# Convert 'waterfront' column to 0's and 1's to represent 'NO' and 'YES'
def map_waterfront_bool(arr):
    if arr == 'YES':
        return 1
    elif arr == 'NO':
        return 0

# Convert 'condition' column into an int64 format from object
def condition_conv(vals):
    if vals == 'Very Good':
        return 5
    elif vals == 'Good':
        return 4
    elif vals == 'Average':
        return 3
    elif vals == 'Fair':
        return 2
    elif vals == 'Poor':
        return 1

# Convert the '?' from "sqft_basement" column to 0 (best to assume they do not have a basement)
def sqft_question(sqft):
    if sqft == '?':
        return 0
    else:
        return sqft

# Convert all 'NONE' views to 0's in order to represent it in a boolean form
def view_bool(views):
    if views == 'NONE':
        return 0
    else:
        return 1

# Convert 'date' column into int64 with year | Unnecessary as we dont really care about the posted date
# Columns for renovation year and built year are much more relevant.
df['date'] = df['date'].str[5:]
df['date'] = df['date'].str.replace('/','').astype(np.int64)

In [83]:
# Call all functions to turn data into a manipulatable format
# First fill missing values in 'waterfront' column to 'NO'
df['waterfront'].fillna('NO', inplace = True)
df['waterfront'] = df['waterfront'].map(map_waterfront_bool)
df['condition'] = df['condition'].map(condition_conv)
df['sqft_basement'] = df['sqft_basement'].map(sqft_question).astype(float)
df['grade'] = df['grade'].str[:2].astype(np.int64)
df['view'] = df['view'].map(view_bool)

# Defining a limitation so Folium doesn't crash this notebook
df_below_par = df[df['condition'] < 2]

In [84]:
import folium

lat = 47.5714
long = -122.191

base_map = folium.Map([lat, long], zoom_start=11)

In [85]:
x = df_below_par['lat']
y = df_below_par['long']

points = list(zip(x, y))
for p in points:
    lat = p[0]
    long = p[1]
    popup_text = "Latitude: {}, Longitude: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)

folium.CircleMarker(
    location=[47.453327, -122.107446],
    radius=50,
    popup="Laurelhurst Park",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(base_map)

folium.CircleMarker(
    radius=50,
    location=[47.344300, -122.279052],
    popup="The Waterfront",
    color="crimson",
    fill=False,
).add_to(base_map)

In [86]:
base_map